In [2]:
import gzip
import json

import numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
fl=gzip.open('goemotions.json.gz')
res=json.load(fl)
#test
print(res[0])

['That game hurt.', 'sadness', 'negative']


In [4]:
df=pd.DataFrame(res,columns=["post","emotion","sentiment"])
emotions=df.drop(columns=['sentiment','post'])
sentiments=df.drop(columns=['emotion','post'])

In [5]:
posts=df.drop(columns=['emotion','sentiment'])
numpy_posts=posts.to_numpy().flatten()
numpy_emotions=emotions.to_numpy().flatten()
numpy_sentiments=sentiments.to_numpy().flatten()

In [6]:
import gensim.downloader as api
embedding=api.load("word2vec-google-news-300")

In [13]:
# import gensim
# from gensim import models
# from gensim.models import Word2Vec
# model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [14]:
print(numpy_posts)

['That game hurt.' "You do right, if you don't care then fuck 'em!"
 'Man I love reddit.' ...
 'Well when you’ve imported about a gazillion of them I or your country it’s gets serious.'
 'That looks amazing'
 "The FDA has plenty to criticize. But like here, it's usually criticized horribly off base. It needs to grow some balls and actually enforce things. "]


In [7]:
import nltk
from nltk import word_tokenize



In [ ]:
nltk.download('punkt')
token_word = []
for words in numpy_posts:
 token_word.extend(word_tokenize(words))

In [23]:
len(token_word)

2642128

In [8]:
posts_embedding=[]
for post in numpy_posts:
 words=word_tokenize(post)
 word_embedding=[]
 for word in words:
   try:
    temp=embedding[word]
    word_embedding.append(temp)
   except KeyError:
    pass
 accumumlate=[0]*300
 accumumlate=np.array(accumumlate)
 for embedding_vec in word_embedding:
         accumumlate=embedding_vec+accumumlate
 if len(word_embedding)!=0:
    accumumlate=accumumlate/len(word_embedding)
    posts_embedding.append(numpy.copy(accumumlate))






In [24]:
print(posts_embedding[1])

[ 6.25732422e-02  3.09539795e-02  5.25146484e-02  1.86047363e-01
 -9.67407227e-02  1.77734375e-02  7.15301514e-02 -4.30419922e-02
 -9.18579102e-03 -9.52987671e-03 -1.21923828e-01 -1.82128906e-01
 -6.42822266e-02  3.07006836e-02 -1.66918945e-01  1.21215820e-01
  6.70410156e-02  9.59472656e-02  7.86376953e-02 -1.34497070e-01
 -1.89697266e-02  1.28784180e-02  1.84210205e-01  1.10565186e-02
  2.64997482e-02  5.17448425e-02 -1.15362549e-01  4.36706543e-02
  1.41555786e-02 -5.31677246e-02 -6.83593750e-02  1.43261719e-01
 -3.82324219e-02 -6.48437500e-02 -1.05448914e-01  4.04418945e-02
  8.79394531e-02  5.19531250e-02  6.40869141e-04  1.19612122e-01
  4.73510742e-02 -4.95117188e-02  2.59423828e-01 -6.33392334e-02
 -1.74804687e-02 -2.39746094e-02  2.94799805e-02 -1.01855469e-01
  1.83105469e-03 -1.88598633e-02 -9.89517212e-02  9.99755859e-02
 -5.35888672e-03  6.20971680e-02  1.24420166e-02 -2.30712891e-03
 -3.41903687e-02 -4.81323242e-02 -8.86230469e-03 -1.30187988e-02
  1.18041992e-02  3.08044

In [39]:
numpy_posts[1]
word_tokenize(numpy_posts[1])

['You',
 'do',
 'right',
 ',',
 'if',
 'you',
 'do',
 "n't",
 'care',
 'then',
 'fuck',
 "'em",
 '!']

In [40]:
print((embedding['You']+embedding['do']+embedding['right']+embedding['if']+embedding['you']+embedding['do']+embedding['care']+embedding['then']+embedding['fuck']+embedding["n't"])/10)

[ 6.25732392e-02  3.09539791e-02  5.25146499e-02  1.86047360e-01
 -9.67407227e-02  1.77734382e-02  7.15301484e-02 -4.30419929e-02
 -9.18579102e-03 -9.52987652e-03 -1.21923827e-01 -1.82128906e-01
 -6.42822236e-02  3.07006836e-02 -1.66918948e-01  1.21215820e-01
  6.70410171e-02  9.59472656e-02  7.86376968e-02 -1.34497076e-01
 -1.89697258e-02  1.28784180e-02  1.84210211e-01  1.10565182e-02
  2.64997482e-02  5.17448410e-02 -1.15362547e-01  4.36706543e-02
  1.41555788e-02 -5.31677231e-02 -6.83593750e-02  1.43261716e-01
 -3.82324234e-02 -6.48437515e-02 -1.05448917e-01  4.04418930e-02
  8.79394561e-02  5.19531257e-02  6.40869141e-04  1.19612120e-01
  4.73510735e-02 -4.95117195e-02  2.59423822e-01 -6.33392334e-02
 -1.74804684e-02 -2.39746086e-02  2.94799805e-02 -1.01855472e-01
  1.83105469e-03 -1.88598633e-02 -9.89517197e-02  9.99755859e-02
 -5.35888691e-03  6.20971695e-02  1.24420170e-02 -2.30712886e-03
 -3.41903679e-02 -4.81323227e-02 -8.86230450e-03 -1.30187990e-02
  1.18041988e-02  3.08044

In [38]:
embedding["'em"]

KeyError: "Key ''em' not present"